In [176]:
from keras.models import load_model
import numpy as np
import pandas as pd
import re
import nltk
import ast

In [177]:
model = load_model('GRU.h5')

In [178]:
df_combined = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Tokenized_Data/df_combined.csv')
df_combined_test = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Tokenized_Data/df_combined_test.csv')

In [179]:
df_test = pd.read_csv("/Users/ved/Desktop/Sem 7/DL Project/Dataset/test_labels.csv")

In [180]:
for i in range(len(df_combined['comment'])):
    df_combined["comment"][i] = ast.literal_eval(df_combined["comment"][i])
for i in range(len(df_combined_test['comment'])):
    df_combined_test["comment"][i] = ast.literal_eval(df_combined_test["comment"][i])

In [181]:
X_train1 = df_combined['comment']
X_test1 = df_combined_test['comment']
max_features = 30000
maxlen = 100

In [182]:
X_test = tokenizer.texts_to_sequences(X_test1)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
y_pred = model.predict(x_test, batch_size=1024)
y_pred = (y_pred > 0.5).astype(int)

63/63 [==============================] - 18s 287ms/step


In [183]:
y_actual = df_combined_test
y_actual.drop(columns="comment",inplace=True)

In [190]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming y_actual contains the true labels and y_predicted contains the predicted labels
# You can replace these with your actual and predicted labels # Replace with your true labels
y_predicted = y_pred # Replace with your predicted labels

# Calculate accuracy
accuracy = accuracy_score(y_actual, y_predicted)

# Calculate precision, recall, and F1-score
precision = precision_score(y_actual, y_predicted,average='micro')
recall = recall_score(y_actual, y_predicted,average='micro')
f1 = f1_score(y_actual, y_predicted,average='micro')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Accuracy: 0.8662196380005627
Precision: 0.5359717248755987
Recall: 0.7949372327217548
F1-score: 0.6402599927779785


In [191]:
input= ["shit"]
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train1) + list(X_test1))
def custom(input):
    input = tokenizer.texts_to_sequences(input)
    input = sequence.pad_sequences(input, maxlen=maxlen)
    pred = model.predict(input)
    toxicity_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    j=0
    for i in pred[0]:
        if i > 0.5:
            pred[0][j]=1
        else:
            pred[0][j]=0
        j = j+1
    for i, class_name in enumerate(toxicity_classes):
        print(f"{class_name}: {pred[0][i]}")

In [192]:
custom(input)

1/1 [==============================] - 0s 14ms/step
toxic: 1.0
severe_toxic: 0.0
obscene: 1.0
threat: 0.0
insult: 0.0
identity_hate: 0.0
